In [1]:
%load_ext autoreload 
%autoreload 2

import os
import sys
import shutil
import numpy as np
import pandas as pd
import touchscreen_toolbox as tb
from touchscreen_toolbox.utils import *

In [46]:
def initialize(folder_path):
    """
    Check progress of a folder,
    initialize folder/files if the folder is unprocessed,
    otherwise return a to-do list for higher level functions.
    """
    
    videos = [f for f in os.listdir(folder_path) if f.endswith(".mp4")]
    if not videos:
        return []
    
    dlc_folder = os.path.join(folder_path, DLC_FOLDER)
    rst_folder = os.path.join(folder_path, RST_FOLDER)
    stats_path = os.path.join(rst_folder, STATS_NAME)
    
    # no previous progress
    if not os.path.exists(stats_path):
        mk_dir(dlc_folder)
        mk_dir(rst_folder)
        STATS_TEMPL.to_csv(stats_path, index=False)
        print(f"Initialized under {folder_path}")
        return videos
    
    # read progress and exclude processed videos
    else:
        analyzed = pd.read_csv(stats_path).iloc[2:, 0].values
        if not os.path.exists(dlc_folder):
            mk_dir(dlc_folder)
        return [v for v in videos if v not in analyzed]

In [39]:
def analyze_video(video_path):
    
    video_name  = os.path.basename(video_path)
    folder_path = os.path.dirname(video_path)
    curr_files  = os.listdir(folder_path)
    
    # preprocess
    # get new video name and a list of process applied (if any)
    new_video, proc_ls = tb.preprocess.preprocess(video_path)

    # analyze
    tb.dlc.dlc_analyze(DLC_CONFIG, new_video)
    new_files = [f for f in os.listdir(folder_path) if f not in curr_files]
    csv = os.path.join(folder_path, [f for f in new_files if f.endswith(".csv")][0])
    
    # postprocess
    # record into stats
    data  = tb.utils.read_dlc_csv(csv)
    
    stats_path = os.path.join(folder_path, RST_FOLDER, STATS_NAME)
    if os.path.exists(stats_path):
        stats = pd.read_csv(stats_path)
        stats.loc[len(data)] = ([video_name] + [str(proc_ls)] + tb.postprocess.statistics(data))
        stats.to_csv(stats_path, index=False, header=False)
    
    # standardize
    data.drop(CCOLS, axis=1)
    tb.postprocess.standardize(data)
    tb.postprocess.fillna(data)
    data.to_csv(os.path.join(folder_path, RST_FOLDER, video_name[:-4] + '.csv'))
    
    # cleanup if folder exists
    dlc_folder = os.path.join(folder_path, DLC_FOLDER)
    if os.path.exists(dlc_folder):
        for f in new_files:
            file_path = os.path.join(folder_path, f)
            new_path = os.path.join(dlc_folder, f)
            os.rename(file_path, new_path)

In [40]:
analyze_video("tests/20220101/2_clip.mp4")

C:\Users\25791\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:21<00:00, 23.81it/s]
503it [00:00, 530.05it/s]
503it [00:00, 1499.01it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


In [41]:
def analyze_folder(folder_path):
    """Analyze individual folder"""
    
    to_analyze = initialize(folder_path)
    
    for video in to_analyze:
        video_path = os.path.join(folder_path, video)
        analyze_video(video_path)

def analyze_folders(root, ):
    """Analyze the folder & all sub-folders"""
    
    for (folder_path, _, files) in list(os.walk(root)):
        analyze_folder(folder_path)

In [47]:
analyze_folders('tests')

Initialized files under tests\20220101


100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:20<00:00, 24.10it/s]
503it [00:00, 540.34it/s]
503it [00:00, 1477.13it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]
C:\Users\25791\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:20<00:00, 24.31it/s]
503it [00:00, 553.61it/s]
503it [00:00, 1429.41it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1998.24it/s]


Increasing brightness for 'tests\20220101\7_clip.mp4'


C:\Users\25791\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:20<00:00, 24.80it/s]
503it [00:00, 541.42it/s]
503it [00:00, 1527.02it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1997.05it/s]


Initialized files under tests\20220102


C:\Users\25791\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:21<00:00, 23.58it/s]
503it [00:00, 531.58it/s]
503it [00:00, 1506.44it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2003.01it/s]
